<a href="https://colab.research.google.com/github/tanyaS121/HeartDiseasePrediction/blob/main/HeartD_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
%matplotlib inline

Loading the dataset

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

Shape of the dataset

In [ ]:
data.shape

Variable types in the dataset

In [ ]:
data.dtypes

##**EDA AND PRE-PROCESSING**


Outcome variable count

In [ ]:
sns.catplot(x='target', kind='count', palette='ch:.25', data=data)

Categorical predictive variables

In [ ]:
sns.catplot(x='sex', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='cp', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='fbs', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='restecg', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='exang', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='slope', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='ca', kind='count', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.catplot(x='thal', kind='count', hue='target', data=data, palette='ch:.25')

Distributional predictive variables

In [ ]:
data[['age','trestbps','chol','thalach','oldpeak']].describe()

In [ ]:
sns.displot(x='age', multiple='stack', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.displot(x='trestbps', multiple='stack', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.displot(x='chol', multiple='stack', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.displot(x='trestbps', multiple='stack', hue='target', data=data, palette='ch:.25')

In [ ]:
sns.displot(x='oldpeak', multiple='stack', hue='target', data=data, palette='ch:.25')

## **SPLITTING AND PRE-PROCESSING**

Defining x_train,x_test,y_train and y_test

In [21]:
x = data.drop('target', axis=1)
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Scaling the data

In [23]:
sc = StandardScaler().fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)

#TRAINING THE MODEL
